## PRODUCT-LENGTH PREDICTION

In [2]:
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [4]:
df_train = pd.read_csv('train.csv')

In [5]:
df_train = df_train.dropna()

In [6]:
df_test = pd.read_csv('test.csv')

In [7]:
df_test = df_test.dropna()

In [8]:
df_train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,"[Simple and elegant, great for displaying indo...",HINS Brings you the most Elegant Looking Pot w...,5725,950.000000
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...,<p><strong>Aluminum Foil Stickers-good kitchen...,6030,984.251967
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,[100% Pure And Natural Essential Oil Or Fragra...,"Transform your home, workplace or hotel room i...",8201,393.700787


In [9]:
df_test.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
5,2782548,Evershine Shoppee 10m Waxed Nylon Thread Cotto...,[Kindly Refer The Product Description Before B...,Description: Material: Nylon Length: about 10m...,11205
11,2736605,SHASAK Sanganer Hand Block Printed Short Kurta...,[Confused between wearing a T-shirt and a shir...,Men’s Pure Cotton Sanganeri Hand-Block Print S...,2911


In [10]:
df_train.isnull().sum()

PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
PRODUCT_LENGTH     0
dtype: int64

In [11]:
df_train.shape 

(1038460, 6)

In [15]:
df_train['TITLE'].value_counts() 

Zeya Yellow Gold Ring                                                                                                                                                          47
Front Optimum Oep Series Brake Pad With Rubber Steel Rubber Shims                                                                                                              29
Irnado Women Leather Comfortable Footwear For All Occasions                                                                                                                    29
Outdoor Patio Furniture Covers Waterproof Rectangular Patio Table Cover, Oxford Polyester Material Windproof, Anti-UV,Durable Waterproof Dustproof Outdoor Cover for Garden    28
Front R1 Concepts Ceramic Series Brake Pad With Rubber Steel Rubber Shims                                                                                                      23
                                                                                                              

In [16]:
df_train['BULLET_POINTS'].value_counts() 

[Good quality and Suitable to use.,This Product comes in a proper Packaging.,In case of any query or issue. Feel free to reach out to us.,Contains: Pack of 1.,Delivery within 3-5 weeks.]                                                                                                                                                                                                                                                                                                                                                                                             10119
[Generic are "Made in India' products using sublimation printing process, with print covering all 3 sides of cover.,Due to the nature of different monitors/tablets/smart phones, color variation may occur and the product may look slightly different in person being a handmade item.,Generic Cases are waterproof, fade resistant, non-peeling, printed with vibrant eco-friendly inks, extremely slim fitting with access to all funct

In [18]:
def join_bullet_points(bullet_points):
    sentence = ". ".join(bullet_points)+"."
    return sentence
df_train['FEATURES'] = df_train['BULLET_POINTS'].apply(lambda x: join_bullet_points(x.split('\n')))

In [19]:
df_train['FEATURES'].head()

2    [Loud Dual Tone Trumpet Horn, Compatible With ...
3    [Made By 95%cotton and 5% Lycra which gives yo...
5    [Simple and elegant, great for displaying indo...
7    [HIGH QUALITY PVC MATERIAL: The kitchen alumin...
9    [100% Pure And Natural Essential Oil Or Fragra...
Name: FEATURES, dtype: object

In [20]:
df_train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,FEATURES
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495,"[Loud Dual Tone Trumpet Horn, Compatible With ..."
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574,[Made By 95%cotton and 5% Lycra which gives yo...
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,"[Simple and elegant, great for displaying indo...",HINS Brings you the most Elegant Looking Pot w...,5725,950.000000,"[Simple and elegant, great for displaying indo..."
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...,<p><strong>Aluminum Foil Stickers-good kitchen...,6030,984.251967,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,[100% Pure And Natural Essential Oil Or Fragra...,"Transform your home, workplace or hotel room i...",8201,393.700787,[100% Pure And Natural Essential Oil Or Fragra...


In [21]:
df_train= df_train.drop('BULLET_POINTS', axis=1)


In [22]:
df_train= df_train.drop('PRODUCT_TYPE_ID', axis=1)

In [23]:
df_train

,PRODUCT_ID,TITLE,DESCRIPTION,PRODUCT_LENGTH,FEATURES
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"Specifications: Color: Red, Material: Aluminiu...",748.031495,"[Loud Dual Tone Trumpet Horn, Compatible With ..."
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,787.401574,[Made By 95%cotton and 5% Lycra which gives yo...
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,HINS Brings you the most Elegant Looking Pot w...,950.000000,"[Simple and elegant, great for displaying indo..."
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,<p><strong>Aluminum Foil Stickers-good kitchen...,984.251967,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,"Transform your home, workplace or hotel room i...",393.700787,[100% Pure And Natural Essential Oil Or Fragra...
...,...,...,...,...,...
2249686,1898841,Cocomii Holographic iPhone 11 Pro Case - Holog...,<b>COCOMII</b><br><br>We transform amazing ide...,600.000000,"[COCOMII - UNIQUE, STYLISH, QUALITY: This beau..."
2249689,1616561,Have It Tall Men's Curved Hem Long Drop Tail T...,This extra long Tall t-Shirt will be your favo...,1200.000000,[Light Weight 65% Polyester/ 35% Ring Spun Cot...
2249694,2766635,"(3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...",<p><b>[Brand]: </b>XVIEONR</p> <p><br></p> <p>...,125.984252,[❤ [Inspiration] Inspired by the Untitled Goos...
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,Wall Clocks Are Very Attractive In Looks And E...,1200.000000,"[Dial size: 12 inches in diameter,Big, clear r..."


In [24]:
pip install bs4

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1266 sha256=5cd27619d843c5b24fe8b9cc7c116e9e0c6cd9e152570792cdac1df3456eb9b3
  Stored in directory: c:\users\abhis\appdata\local\pip\cache\wheels\25\42\45\b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [25]:
from bs4 import BeautifulSoup


In [27]:
import re
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(lambda x: remove_tags(x))

In [28]:
df_train

,PRODUCT_ID,TITLE,DESCRIPTION,PRODUCT_LENGTH,FEATURES
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"Specifications: Color: Red, Material: Aluminiu...",748.031495,"[Loud Dual Tone Trumpet Horn, Compatible With ..."
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,787.401574,[Made By 95%cotton and 5% Lycra which gives yo...
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,HINS Brings you the most Elegant Looking Pot w...,950.000000,"[Simple and elegant, great for displaying indo..."
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,Aluminum Foil Stickers-good kitchen helper for...,984.251967,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,"Transform your home, workplace or hotel room i...",393.700787,[100% Pure And Natural Essential Oil Or Fragra...
...,...,...,...,...,...
2249686,1898841,Cocomii Holographic iPhone 11 Pro Case - Holog...,COCOMIIWe transform amazing ideas and advanced...,600.000000,"[COCOMII - UNIQUE, STYLISH, QUALITY: This beau..."
2249689,1616561,Have It Tall Men's Curved Hem Long Drop Tail T...,This extra long Tall t-Shirt will be your favo...,1200.000000,[Light Weight 65% Polyester/ 35% Ring Spun Cot...
2249694,2766635,"(3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...",[Brand]: XVIEONR [Product name]: Fashion cute...,125.984252,[❤ [Inspiration] Inspired by the Untitled Goos...
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,Wall Clocks Are Very Attractive In Looks And E...,1200.000000,"[Dial size: 12 inches in diameter,Big, clear r..."


In [29]:
def join_sentence(arr):
    return ' '.join(arr)

df_train['FEATURES'] = df_train['FEATURES'].apply(lambda x: join_sentence(x))


In [30]:
df_train

,PRODUCT_ID,TITLE,DESCRIPTION,PRODUCT_LENGTH,FEATURES
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"Specifications: Color: Red, Material: Aluminiu...",748.031495,[ L o u d D u a l T o n e T r u m p e t ...
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,787.401574,[ M a d e B y 9 5 % c o t t o n a n d ...
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,HINS Brings you the most Elegant Looking Pot w...,950.000000,"[ S i m p l e a n d e l e g a n t , g r ..."
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,Aluminum Foil Stickers-good kitchen helper for...,984.251967,[ H I G H Q U A L I T Y P V C M A T E R ...
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,"Transform your home, workplace or hotel room i...",393.700787,[ 1 0 0 % P u r e A n d N a t u r a l ...
...,...,...,...,...,...
2249686,1898841,Cocomii Holographic iPhone 11 Pro Case - Holog...,COCOMIIWe transform amazing ideas and advanced...,600.000000,"[ C O C O M I I - U N I Q U E , S T Y L ..."
2249689,1616561,Have It Tall Men's Curved Hem Long Drop Tail T...,This extra long Tall t-Shirt will be your favo...,1200.000000,[ L i g h t W e i g h t 6 5 % P o l y e ...
2249694,2766635,"(3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...",[Brand]: XVIEONR [Product name]: Fashion cute...,125.984252,[ ❤ [ I n s p i r a t i o n ] I n s p i r ...
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,Wall Clocks Are Very Attractive In Looks And E...,1200.000000,[ D i a l s i z e : 1 2 i n c h e s i ...


In [31]:
temp_train = pd.read_csv("train.csv")

In [32]:
df_train['FEATURES'] = temp_train['FEATURES']

KeyError: 'FEATURES'

In [33]:
temp_train.dropna

<bound method DataFrame.dropna of          PRODUCT_ID                                              TITLE  \
0           1925202  ArtzFolio Tulip Flowers Blackout Curtain for D...   
1           2673191  Marks & Spencer Girls' Pyjama Sets T86_2561C_N...   
2           2765088  PRIKNIK Horn Red Electric Air Horn Compressor ...   
3           1594019  ALISHAH Women's Cotton Ankle Length Leggings C...   
4            283658  The United Empire Loyalists: A Chronicle of th...   
...             ...                                                ...   
2249693     2422167  Nike Women's As W Ny Df Swsh Hn Kh Bra (CZ7610...   
2249694     2766635  (3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...   
2249695     1987786  Kangroo Sweep Movement Printed Wooden Wall Clo...   
2249696     1165754  Electro Voice EKX-BRKT15 | Wall Mount Bracket ...   
2249697     1072666  Skyjacker C7360SP Component Box For PN[C7360PK...   

                                             BULLET_POINTS  \
0        [LUXUR

In [34]:
temp_train = temp_train.dropna()

In [35]:
df_train['FEATURES'] = temp_train['FEATURES']

KeyError: 'FEATURES'

In [36]:
df_train['FEATURES'] = temp_train['BULLET_POINTS']

In [37]:
df_train

,PRODUCT_ID,TITLE,DESCRIPTION,PRODUCT_LENGTH,FEATURES
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"Specifications: Color: Red, Material: Aluminiu...",748.031495,"[Loud Dual Tone Trumpet Horn, Compatible With ..."
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,787.401574,[Made By 95%cotton and 5% Lycra which gives yo...
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,HINS Brings you the most Elegant Looking Pot w...,950.000000,"[Simple and elegant, great for displaying indo..."
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,Aluminum Foil Stickers-good kitchen helper for...,984.251967,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,"Transform your home, workplace or hotel room i...",393.700787,[100% Pure And Natural Essential Oil Or Fragra...
...,...,...,...,...,...
2249686,1898841,Cocomii Holographic iPhone 11 Pro Case - Holog...,COCOMIIWe transform amazing ideas and advanced...,600.000000,"[COCOMII - UNIQUE, STYLISH, QUALITY: This beau..."
2249689,1616561,Have It Tall Men's Curved Hem Long Drop Tail T...,This extra long Tall t-Shirt will be your favo...,1200.000000,[Light Weight 65% Polyester/ 35% Ring Spun Cot...
2249694,2766635,"(3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...",[Brand]: XVIEONR [Product name]: Fashion cute...,125.984252,[❤ [Inspiration] Inspired by the Untitled Goos...
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,Wall Clocks Are Very Attractive In Looks And E...,1200.000000,"[Dial size: 12 inches in diameter,Big, clear r..."


In [39]:
df_train['TITLE'] = df_train['TITLE'].str.replace("\r", " ")
df_train['TITLE'] = df_train['TITLE'].str.replace("\n", " ")
df_train['TITLE'] = df_train['TITLE'].str.replace("    ", " ")
df_train['TITLE'] = df_train['TITLE'].str.replace('"', '')
df_train['TITLE'] = df_train['TITLE'].str.lower()

In [40]:
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].str.replace("\r", " ")
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].str.replace("\n", " ")
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].str.replace("    ", " ")
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].str.replace('"', '')
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].str.lower()

In [41]:
df_train.head()

,PRODUCT_ID,TITLE,DESCRIPTION,PRODUCT_LENGTH,FEATURES
2,2765088,priknik horn red electric air horn compressor ...,"specifications: color: red, material: aluminiu...",748.031495,"[Loud Dual Tone Trumpet Horn, Compatible With ..."
3,1594019,alishah women's cotton ankle length leggings c...,aishah women's lycra cotton ankel leggings. br...,787.401574,[Made By 95%cotton and 5% Lycra which gives yo...
5,2152929,hins metal bucket shape plant pot for indoor &...,hins brings you the most elegant looking pot w...,950.000000,"[Simple and elegant, great for displaying indo..."
7,2026580,delavala self adhesive kitchen backsplash wall...,aluminum foil stickers-good kitchen helper for...,984.251967,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...
9,2998633,hexwell essential oil for home fragrance oil a...,"transform your home, workplace or hotel room i...",393.700787,[100% Pure And Natural Essential Oil Or Fragra...


In [42]:
df_test


,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
5,2782548,Evershine Shoppee 10m Waxed Nylon Thread Cotto...,[Kindly Refer The Product Description Before B...,Description: Material: Nylon Length: about 10m...,11205
11,2736605,SHASAK Sanganer Hand Block Printed Short Kurta...,[Confused between wearing a T-shirt and a shir...,Men’s Pure Cotton Sanganeri Hand-Block Print S...,2911
...,...,...,...,...,...
734728,1805278,E RELAX Arm Blaster-Heavy Duty Padded Thick Ga...,"[We offer you quality material, durable straps...",E RELAX arm blaster is made from superior grad...,12955
734731,921419,"Casual Canine Basic Hoodie for Dogs, 16"" Mediu...",[Brightly colored pet sweatshirts with authent...,"ZA6015 16 43 Size-See Chart Below: Medium: 16""...",7073
734733,841529,Axor 39135001 Citterio Widespread Faucet with ...,"[8-Inch centers,1/2-Inch IPS inlets,9-Inch spo...",39135001 Features: -ADA compliant. -Includes p...,10645
734734,1190194,Caroline's Treasures BB1801DS812 Halloween Bas...,"[Indoor or outdoor aluminum artwork prints,8 i...",Features. Great for inside or outside these Al...,12680


In [43]:
df_test['FEATURE']= df_test['BULLET_POINTS']
df_test= df_test.drop('BULLET_POINTS', axis=1)
df_test= df_test.drop('PRODUCT_TYPE_ID', axis=1)

import re
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

df_test['DESCRIPTION'] = df_test['DESCRIPTION'].apply(lambda x: remove_tags(x))

df_test['TITLE'] = df_test['TITLE'].str.replace("\r", " ")
df_test['TITLE'] = df_test['TITLE'].str.replace("\n", " ")
df_test['TITLE'] = df_test['TITLE'].str.replace("    ", " ")
df_test['TITLE'] = df_test['TITLE'].str.replace('"', '')
df_test['TITLE'] = df_test['TITLE'].str.lower()

df_test['DESCRIPTION'] = df_test['DESCRIPTION'].str.replace("\r", " ")
df_test['DESCRIPTION'] = df_test['DESCRIPTION'].str.replace("\n", " ")
df_test['DESCRIPTION'] = df_test['DESCRIPTION'].str.replace("    ", " ")
df_test['DESCRIPTION'] = df_test['DESCRIPTION'].str.replace('"', '')
df_test['DESCRIPTION'] = df_test['DESCRIPTION'].str.lower()

In [44]:
df_train


,PRODUCT_ID,TITLE,DESCRIPTION,PRODUCT_LENGTH,FEATURES
2,2765088,priknik horn red electric air horn compressor ...,"specifications: color: red, material: aluminiu...",748.031495,"[Loud Dual Tone Trumpet Horn, Compatible With ..."
3,1594019,alishah women's cotton ankle length leggings c...,aishah women's lycra cotton ankel leggings. br...,787.401574,[Made By 95%cotton and 5% Lycra which gives yo...
5,2152929,hins metal bucket shape plant pot for indoor &...,hins brings you the most elegant looking pot w...,950.000000,"[Simple and elegant, great for displaying indo..."
7,2026580,delavala self adhesive kitchen backsplash wall...,aluminum foil stickers-good kitchen helper for...,984.251967,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...
9,2998633,hexwell essential oil for home fragrance oil a...,"transform your home, workplace or hotel room i...",393.700787,[100% Pure And Natural Essential Oil Or Fragra...
...,...,...,...,...,...
2249686,1898841,cocomii holographic iphone 11 pro case - holog...,cocomiiwe transform amazing ideas and advanced...,600.000000,"[COCOMII - UNIQUE, STYLISH, QUALITY: This beau..."
2249689,1616561,have it tall men's curved hem long drop tail t...,this extra long tall t-shirt will be your favo...,1200.000000,[Light Weight 65% Polyester/ 35% Ring Spun Cot...
2249694,2766635,"(3pcs) goose game cute cartoon enamel pins, fu...",[brand]: xvieonr [product name]: fashion cute...,125.984252,[❤ [Inspiration] Inspired by the Untitled Goos...
2249695,1987786,kangroo sweep movement printed wooden wall clo...,wall clocks are very attractive in looks and e...,1200.000000,"[Dial size: 12 inches in diameter,Big, clear r..."


In [45]:
df_test

,PRODUCT_ID,TITLE,DESCRIPTION,FEATURE
1,1729783,dcgaring microfiber throw blanket warm fuzzy p...,dcgaring throw blanket size chart ( w x l ): t...,[QUALITY GUARANTEED: Luxury cozy plush polyest...
2,1871949,i-match auto parts front license plate bracket...,replacement for the following vehicles:2020 le...,"[Front License Plate Bracket Made Of Plastic,D..."
3,1107571,pinmart gold plated excellence in service 1 ye...,our excellence in service lapel pins feature a...,[Available as a single item or bulk packed. Se...
5,2782548,evershine shoppee 10m waxed nylon thread cotto...,description: material: nylon length: about 10m...,[Kindly Refer The Product Description Before B...
11,2736605,shasak sanganer hand block printed short kurta...,men’s pure cotton sanganeri hand-block print s...,[Confused between wearing a T-shirt and a shir...
...,...,...,...,...
734728,1805278,e relax arm blaster-heavy duty padded thick ga...,e relax arm blaster is made from superior grad...,"[We offer you quality material, durable straps..."
734731,921419,"casual canine basic hoodie for dogs, 16 medium...",za6015 16 43 size-see chart below: medium: 16 ...,[Brightly colored pet sweatshirts with authent...
734733,841529,axor 39135001 citterio widespread faucet with ...,39135001 features: -ada compliant. -includes p...,"[8-Inch centers,1/2-Inch IPS inlets,9-Inch spo..."
734734,1190194,caroline's treasures bb1801ds812 halloween bas...,features. great for inside or outside these al...,"[Indoor or outdoor aluminum artwork prints,8 i..."


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [47]:
df_train.drop_duplicates(subset=['TITLE'], inplace=True)

In [48]:
df_train.fillna('', inplace=True)
df_test.fillna('', inplace=True)

In [55]:
df_test2 = pd.read_csv('test.csv')

In [61]:
df_test2

,PRODUCT_ID,TITLE,DESCRIPTION,FEATURE
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,[QUALITY GUARANTEED: Luxury cozy plush polyest...
2,1871949,I-Match Auto Parts Front License Plate Bracket...,Replacement for The Following Vehicles:2020 LE...,"[Front License Plate Bracket Made Of Plastic,D..."
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,Our Excellence in Service Lapel Pins feature a...,[Available as a single item or bulk packed. Se...
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN
...,...,...,...,...
734731,921419,"Casual Canine Basic Hoodie for Dogs, 16"" Mediu...","ZA6015 16 43 Size-See Chart Below: Medium: 16""...",[Brightly colored pet sweatshirts with authent...
734732,2456362,Dive Log Book: Scuba Diving Logbook for Beginn...,NaN,NaN
734733,841529,Axor 39135001 Citterio Widespread Faucet with ...,39135001 Features: -ADA compliant. -Includes p...,"[8-Inch centers,1/2-Inch IPS inlets,9-Inch spo..."
734734,1190194,Caroline's Treasures BB1801DS812 Halloween Bas...,Features. Great for inside or outside these Al...,"[Indoor or outdoor aluminum artwork prints,8 i..."


In [62]:
# Split the data into training and validation sets
X_train = df_train[['TITLE', 'DESCRIPTION', 'FEATURES']]
y_train = df_train['PRODUCT_LENGTH']
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [64]:
df_test

,PRODUCT_ID,TITLE,DESCRIPTION,FEATURE,PRODUCT_LENGTH
1,1729783,dcgaring microfiber throw blanket warm fuzzy p...,dcgaring throw blanket size chart ( w x l ): t...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,-21335.353369
2,1871949,i-match auto parts front license plate bracket...,replacement for the following vehicles:2020 le...,"[Front License Plate Bracket Made Of Plastic,D...",-24944.292547
3,1107571,pinmart gold plated excellence in service 1 ye...,our excellence in service lapel pins feature a...,[Available as a single item or bulk packed. Se...,39923.703781
5,2782548,evershine shoppee 10m waxed nylon thread cotto...,description: material: nylon length: about 10m...,[Kindly Refer The Product Description Before B...,33119.762908
11,2736605,shasak sanganer hand block printed short kurta...,men’s pure cotton sanganeri hand-block print s...,[Confused between wearing a T-shirt and a shir...,-29963.460055
...,...,...,...,...,...
734728,1805278,e relax arm blaster-heavy duty padded thick ga...,e relax arm blaster is made from superior grad...,"[We offer you quality material, durable straps...",-874519.581420
734731,921419,"casual canine basic hoodie for dogs, 16 medium...",za6015 16 43 size-see chart below: medium: 16 ...,[Brightly colored pet sweatshirts with authent...,9493.892033
734733,841529,axor 39135001 citterio widespread faucet with ...,39135001 features: -ada compliant. -includes p...,"[8-Inch centers,1/2-Inch IPS inlets,9-Inch spo...",166774.290127
734734,1190194,caroline's treasures bb1801ds812 halloween bas...,features. great for inside or outside these al...,"[Indoor or outdoor aluminum artwork prints,8 i...",6443.144059


In [65]:
df_test2

,PRODUCT_ID,TITLE,DESCRIPTION,FEATURE
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,[QUALITY GUARANTEED: Luxury cozy plush polyest...
2,1871949,I-Match Auto Parts Front License Plate Bracket...,Replacement for The Following Vehicles:2020 LE...,"[Front License Plate Bracket Made Of Plastic,D..."
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,Our Excellence in Service Lapel Pins feature a...,[Available as a single item or bulk packed. Se...
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN
...,...,...,...,...
734731,921419,"Casual Canine Basic Hoodie for Dogs, 16"" Mediu...","ZA6015 16 43 Size-See Chart Below: Medium: 16""...",[Brightly colored pet sweatshirts with authent...
734732,2456362,Dive Log Book: Scuba Diving Logbook for Beginn...,NaN,NaN
734733,841529,Axor 39135001 Citterio Widespread Faucet with ...,39135001 Features: -ADA compliant. -Includes p...,"[8-Inch centers,1/2-Inch IPS inlets,9-Inch spo..."
734734,1190194,Caroline's Treasures BB1801DS812 Halloween Bas...,Features. Great for inside or outside these Al...,"[Indoor or outdoor aluminum artwork prints,8 i..."


In [63]:
# Extract features from text data
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train['FEATURES'])
X_val_counts = vectorizer.transform(X_val['FEATURES'])
X_test_counts = vectorizer.transform(df_test2['FEATURE'])

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [51]:
# Select a suitable machine learning model
model = LinearRegression()

In [52]:
# Train the model
model.fit(X_train_counts, y_train)

LinearRegression()

In [53]:
# Evaluate the performance of the model on the validation set
y_pred = model.predict(X_val_counts)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE on validation set: {rmse:.2f}")

RMSE on validation set: 461770.53


In [54]:
# Test the model and save predictions to a CSV file
y_test_pred = model.predict(X_test_counts)
df_test['PRODUCT_LENGTH'] = y_test_pred
df_test[['PRODUCT_ID', 'PRODUCT_LENGTH']].to_csv('submission.csv', index=False)